# Predicting House Sale Prices

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

df = pd.read_csv('AmesHousing.tsv', delimiter='\t')

In [2]:
def transform_features(df):
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePraice']]

def train_and_test(df):
    train = df.iloc[0:1460]
    test = df.iloc[1460:]
    lr = LinearRegression()
    
    numerical_train = train.select_dtypes(include=["int", "float"])
    numerical_test = test.select_dtypes(include=["int", "float"])
    
    features = numerical_train.drop("SalePrice")
    lr.fit(numerical_train[features], numerical_train["SalePrice"])
    
    predictions = lr.predict(numerical_test[features])
    mse = mean_squared_error(numerical_test["SalePrice"], predictions)
    rmse = mse ** (1/2)
    return rmse

# Feature Engineering

欠損値を以下の方法で処理する。
1. All columns:Drop any with 5% or more missing values for now.
2. Text columns:
Drop any with 1 or more missing values for now.
3. Numerical columns:
For columns with missing values, fill in with the most common value in that column, that is, mode

In [3]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Frontage       2440 non-null float64
Lot Area           2930 non-null int64
Street             2930 non-null object
Alley              198 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         29

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


1.All columns: Drop any with 5% or more missing values for now.

In [4]:
num_missing = df.isnull().sum()

In [5]:
drop_columns = num_missing[num_missing > len(df)/20].sort_values()

df = df.drop(drop_columns.index, axis=1)

2.Text columns: Drop any with 1 or more missing values for now.

In [6]:
text_num_missing = df.select_dtypes(include=["object"]).isnull().sum()

drop_text_columns = text_num_missing[text_num_missing > 0].sort_values()

df = df.drop(drop_text_columns.index, axis=1)

3.Numerical columns: For columns with missing values, fill in with the most common value in that column, that is, mode.

In [7]:
numerical_num_missing = df.select_dtypes(include=["int", "float"]).isnull().sum()

fix_numerical_cols = numerical_num_missing[numerical_num_missing > 0].sort_values(ascending=False)
fix_numerical_cols

Mas Vnr Area      23
Bsmt Half Bath     2
Bsmt Full Bath     2
Garage Area        1
Garage Cars        1
Total Bsmt SF      1
Bsmt Unf SF        1
BsmtFin SF 2       1
BsmtFin SF 1       1
dtype: int64

In [8]:
replacement_values_dict = df[fix_numerical_cols.index].mode().to_dict(orient="records")[0]
replacement_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [9]:
df = df.fillna(replacement_values_dict)

In [10]:
df.isnull().sum().value_counts()

0    64
dtype: int64

4.Create new columns.

In [11]:
years_sold = df["Yr Sold"] - df["Year Built"]
print(years_sold[years_sold < 0])

2180   -1
dtype: int64


In [12]:
years_since_remod = df["Yr Sold"] - df["Year Remod/Add"]
print(years_since_remod[years_since_remod < 0])

1702   -1
2180   -2
2181   -1
dtype: int64


In [13]:
df["Years Before Sale"] = years_sold
df["Years Since Remod"] = years_since_remod

df = df.drop([1702, 2180, 2181], axis=0)

df = df.drop(["Year Built", "Year Remod/Add"], axis=1)

5.Drop the columns that aren't usuful for machine learning, and that leak data about the final sale, refered [the data documentation](https://s3.amazonaws.com/dq-content/307/data_description.txt)

In [14]:
#not usuful for machine learning
df = df.drop(["Order","PID"], axis=1)

#leak data about the final sale
df = df.drop(["Mo Sold", "Yr Sold", "Sale Condition", "Sale Type"], axis=1)

`transform_features`を更新する

In [15]:
def transform_features(df):
    num_missing = df.isnull().sum()
    
    drop_columns = num_missing[num_missing > len(df)/20].sort_values()
    df = df.drop(drop_columns.index, axis=1)
    
    text_num_missing = df.select_dtypes(include=["object"]).isnull().sum()
    drop_text_columns = text_num_missing[text_num_missing > 0].sort_values()
    df = df.drop(drop_text_columns.index, axis=1)
    
    numerical_num_missing = df.select_dtypes(include=["int", "float"]).isnull().sum()
    fix_numerical_cols = numerical_num_missing[numerical_num_missing > 0].sort_values(ascending=False)
    replacement_values_dict = df[fix_numerical_cols.index].mode().to_dict(orient="records")[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df["Yr Sold"] - df["Year Built"]
    years_since_remod = df["Yr Sold"] - df["Year Remod/Add"]
    
    df["Years Before Sale"] = years_sold
    df["Years Since Remod"] = years_since_remod

    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["Year Built", "Year Remod/Add"], axis=1)
    
    #not usuful for machine learning
    df = df.drop(["Order","PID"], axis=1)

    #leak data about the final sale
    df = df.drop(["Mo Sold", "Yr Sold", "Sale Condition", "Sale Type"], axis=1)
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df.iloc[0:1460]
    test = df.iloc[1460:]
    lr = LinearRegression()
    
    numerical_train = train.select_dtypes(include=["int", "float"])
    numerical_test = test.select_dtypes(include=["int", "float"])
    
    features = numerical_train.columns.drop("SalePrice")
    lr.fit(numerical_train[features], numerical_train["SalePrice"])
    
    predictions = lr.predict(numerical_test[features])
    mse = mean_squared_error(numerical_test["SalePrice"], predictions)
    rmse = mse ** (1/2)
    return rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.36731241307

# Feature Selection

In [16]:
numerical_df = transform_df.select_dtypes(include=["int", "float"])
numerical_df

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years Before Sale,Years Since Remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,212,34,0,0,0,0,0,189900,13,12
5,60,9978,6,6,20.0,602.0,0.0,324.0,926.0,926,...,360,36,0,0,0,0,0,195500,12,12
6,120,4920,8,5,0.0,616.0,0.0,722.0,1338.0,1338,...,0,0,170,0,0,0,0,213500,9,9
7,120,5005,8,5,0.0,263.0,0.0,1017.0,1280.0,1280,...,0,82,0,0,144,0,0,191500,18,18
8,120,5389,8,5,0.0,1180.0,0.0,415.0,1595.0,1616,...,237,152,0,0,0,0,0,236500,15,14
9,60,7500,7,5,0.0,0.0,0.0,994.0,994.0,1028,...,140,60,0,0,0,0,0,189000,11,11


In [17]:
abs_corr_coeffs = numerical_df.corr()["SalePrice"].abs().sort_values()
abs_corr_coeffs

BsmtFin SF 2         0.006127
Misc Val             0.019273
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: Sale

相関係数|R| > 0.4のコラムを残す

In [18]:
abs_corr_coeffs[abs_corr_coeffs > 0.4]

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [19]:
transform_df = transform_df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, axis=1)

In [20]:
# Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [21]:
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)
        
uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
uniqueness_counts

Street           2
Central Air      2
Land Contour     4
Lot Config       5
Bldg Type        5
Roof Style       6
Foundation       6
Heating          6
MS Zoning        7
Condition 2      8
House Style      8
Roof Matl        8
Condition 1      9
Exterior 1st    16
Exterior 2nd    17
Neighborhood    28
dtype: int64

In [22]:
# カテゴリー数が10以下の項目のみを扱う
drop_uni_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_df = transform_df.drop(drop_uni_cols, axis=1)

In [23]:
#　カテゴリー型に
text_cols = transform_df.select_dtypes(include=["object"])
for col in text_cols:
    transform_df[col] = transform_df[col].astype("category")
    
transform_df = pd.concat([
    transform_df,
    pd.get_dummies(transform_df.select_dtypes(include=["category"]))
]).drop(text_cols, axis=1)

`select_features`を更新する

In [24]:
def transform_features(df):
    num_missing = df.isnull().sum()
    
    drop_columns = num_missing[num_missing > len(df)/20].sort_values()
    df = df.drop(drop_columns.index, axis=1)
    
    text_num_missing = df.select_dtypes(include=["object"]).isnull().sum()
    drop_text_columns = text_num_missing[text_num_missing > 0].sort_values()
    df = df.drop(drop_text_columns.index, axis=1)
    
    numerical_num_missing = df.select_dtypes(include=["int", "float"]).isnull().sum()
    fix_numerical_cols = numerical_num_missing[numerical_num_missing > 0].sort_values(ascending=False)
    replacement_values_dict = df[fix_numerical_cols.index].mode().to_dict(orient="records")[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df["Yr Sold"] - df["Year Built"]
    years_since_remod = df["Yr Sold"] - df["Year Remod/Add"]
    
    df["Years Before Sale"] = years_sold
    df["Years Since Remod"] = years_since_remod

    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["Year Built", "Year Remod/Add"], axis=1)
    
    #not usuful for machine learning
    df = df.drop(["Order","PID"], axis=1)

    #leak data about the final sale
    df = df.drop(["Mo Sold","Yr Sold", "Sale Condition", "Sale Type"], axis=1)
    return df

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return df


def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse
    
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

[26039.457907030224, 25635.488823740623, 36989.36621438267, 28421.324999546123]


29271.40948617491